In [1]:
from pinecone import Pinecone, ServerlessSpec

from transformers import CLIPProcessor, CLIPModel
import os
import torch
import base64
import pandas as pd
import io

from dotenv import load_dotenv


import sys 

sys.path.append("..")
sys.path.append("../utlities")

from utlities.directories import  IMAGES_FOLDER


from IPython import display 
from PIL import Image
import pillow_avif


load_dotenv()


True

In [2]:
# make pinecone index

pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

CLIP_INDEX_NAME = "clip-email-index"

if not pc.has_index(CLIP_INDEX_NAME):
  pc.create_index(
      name=CLIP_INDEX_NAME,
      dimension=512,
      metric="cosine",
      spec=ServerlessSpec(
          cloud='aws',
          region='us-east-1'
      )
  )

index = pc.Index(CLIP_INDEX_NAME)


In [3]:
# make image embeddings DF

model_id = "openai/clip-vit-base-patch32"

processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id)
model.to("cpu")

def create_image_embeddings(image):
    if image is None:
        return None
        
    vals = processor(
          text=[""],
          images=image,
          return_tensors='pt')['pixel_values'].to("cpu")
    image_embedding = model.get_image_features(vals)
    return image_embedding[0]    


def open_image(image_name):
    return Image.open(os.path.join(IMAGES_FOLDER, image_name)) 


images = os.listdir(IMAGES_FOLDER)
images = [im for im in images if not im.startswith(".")]
 
images_df = pd.DataFrame({
    "image_filenames": images
})

images_df['image'] = images_df['image_filenames'].apply(open_image)
images_df['image_embeddings'] = images_df['image'].apply(create_image_embeddings)
 

/Users/charlienatoli/coding_projects/email_content_search/.venv/lib/python3.11/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [4]:
# create and upsert vectors

vectors = []

for i in range(0, images_df.shape[0]):
    d = images_df.loc[i]
    if d['image'] is None:
        continue
    vectors.append({
      "id": d["image_filenames"],
      "values": d["image_embeddings"],
      "metadata": {"caption": ""}
  })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 315}